In [1]:
import os
import pandas as pd
import sys
sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/mu529/Bus-Capstone')
import ttools #homemade module
import gtfs #homemade module
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')

In [2]:
# get the sample of parsed AVL data.  Beware, large files take more time.
bustime = pd.read_csv('spark_parse/test2.txt',header=None)

/opt/rh/anaconda/root/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
# beware!  Bonan is still working on organizing the extract files.  these columns may change.
bustime.columns = ['route','lat','lon','timestamp','vehicle_id','trip_id','trip_date','shape_id','next_stop_id','dist_from_stop','stop_dist_on_trip','presentable_dist']

In [4]:
bustime.drop_duplicates(['vehicle_id','timestamp'],inplace=True)
bustime['trip_id'] = bustime['trip_id'].str.replace('MTA NYCT_','')
bustime['trip_id'] = bustime['trip_id'].str.replace('MTABC_','')
bustime.set_index(['route','trip_id','trip_date','vehicle_id'],inplace=True,drop=True)#,verify_integrity=True)

In [5]:
# for demonstration, use a subset. Just get data for one line (M5) on one day.
tripDateLookup = "2015-01-06"
lineLookup = "MTA NYCT_B41"
bustime = bustime.xs((lineLookup,tripDateLookup),level=(0,2),
                           drop_level=False)
# note that the AVL dataframe must be sorted by timestammp, since iloc[]
# selection is used later in this script to find the earliest time
bustime.sort_index(inplace=True)
bustime['timestamp'] = bustime['timestamp'].apply(ttools.parseActualTime,tdate='2015-01-06')
print 'Finished loading BusTime data, parsing timestamps, and and slicing B41 line.'

Finished loading BusTime data, parsing timestamps, and and slicing B41 line.


In [6]:
# be careful - looks like pandas did not recognize this column to be numeric data
print type(bustime.iloc[1].stop_dist_on_trip)
bustime['stop_dist_on_trip'] = bustime['stop_dist_on_trip'].convert_objects(convert_numeric=True)
bustime['dist_from_stop'] = bustime['dist_from_stop'].convert_objects(convert_numeric=True)

<type 'str'>


In [7]:
bustime['veh_dist_along_trip'] = bustime['stop_dist_on_trip'] - bustime['dist_from_stop']

## Problem: multiple vehicles serving the same trip? 
Usually there is a clear "winner."  Perhaps this due to logging in/out of vehicles before the trip actually begins.

In [22]:
bg = bustime.groupby(level=(0,1,2,3))
trip_veh_validation = pd.DataFrame(bg.size(),columns=['N'])
trip_veh_validation['time_range'] = bg['timestamp'].max()-bg['timestamp'].min()
trip_veh_validation['dist_range'] = bg['veh_dist_along_trip'].max()-bg['veh_dist_along_trip'].min()

In [23]:
trip_veh_validation

N  \
route        trip_id                           trip_date  vehicle_id          
MTA NYCT_B41 FB_A5-Weekday-SDon-004500_B41_1   2015-01-06 MTA NYCT_4233  51   
                                                          MTA NYCT_4615   4   
                                                          MTA NYCT_5049  10   
             FB_A5-Weekday-SDon-006200_B41_2   2015-01-06 MTA NYCT_4615  54   
                                                          MTA NYCT_5049   3   
             FB_A5-Weekday-SDon-009000_B41_1   2015-01-06 MTA NYCT_4233  39   
             FB_A5-Weekday-SDon-009700_B41_3   2015-01-06 MTA NYCT_4239  50   
             FB_A5-Weekday-SDon-012000_B41_2   2015-01-06 MTA NYCT_4615  65   
                                                          MTA NYCT_4955  70   
             FB_A5-Weekday-SDon-012200_B41_4   2015-01-06 MTA NYCT_4244  39   
             FB_A5-Weekday-SDon-015000_B41_3   2015-01-06 MTA NYCT_4239  44   
             FB_A5-Weekday-SDon-015500_B41_1   2015-01-06 MTA NYCT_4233  85   
             FB_A5-Weekday-SDon-018000_B41_4   2015-01-06 MTA NYCT_4244  40   
                                                          MTA NYCT_4955   4   
             FB_A5-Weekday-SDon-018200_B41_2   2015-01-06 MTA NYCT_4615  72   
             FB_A5-Weekday-SDon-021000_B41_1   2015-01-06 MTA NYCT_4233  41   
             FB_A5-Weekday-SDon-021500_B41_3   2015-01-06 MTA NYCT_4239  47   
             FB_A5-Weekday-SDon-024000_B41_2   2015-01-06 MTA NYCT_4229   8   
                                                          MTA NYCT_4233   2   
                                                          MTA NYCT_4615  54   
             FB_A5-Weekday-SDon-024200_B41_4   2015-01-06 MTA NYCT_4244  55   
             FB_A5-Weekday-SDon-025700_B44_3   2015-01-06 MTA NYCT_4225  27   
             FB_A5-Weekday-SDon-026000_B41_5   2015-01-06 MTA NYCT_4225   3   
                                                          MTA NYCT_4229  57   
                                                          MTA NYCT_4233   1   
                                                          MTA NYCT_4955   1   
             FB_A5-Weekday-SDon-027000_B41_3   2015-01-06 MTA NYCT_4239  56   
                                                          MTA NYCT_5176   1   
             FB_A5-Weekday-SDon-027200_B41_7   2015-01-06 MTA NYCT_4235  65   
             FB_A5-Weekday-SDon-028900_B41_9   2015-01-06 MTA NYCT_4963  60   
...                                                                      ..   
             FB_A5-Weekday-SDon-135600_B46_206 2015-01-06 MTA NYCT_4574  63   
             FB_A5-Weekday-SDon-136000_B41_77  2015-01-06 MTA NYCT_5127  74   
             FB_A5-Weekday-SDon-137200_B41_83  2015-01-06 MTA NYCT_5052  67   
             FB_A5-Weekday-SDon-137200_B41_84  2015-01-06 MTA NYCT_4593  60   
             FB_A5-Weekday-SDon-138400_B46_204 2015-01-06 MTA NYCT_4962  57   
             FB_A5-Weekday-SDon-139600_B46_210 2015-01-06 MTA NYCT_5141  70   
             FB_A5-Weekday-SDon-139700_B46_200 2015-01-06 MTA NYCT_4589  80   
                                                          MTA NYCT_5010  23   
             FB_A5-Weekday-SDon-140800_B41_78  2015-01-06 MTA NYCT_4952  19   
             FB_A5-Weekday-SDon-140900_B41_85  2015-01-06 MTA NYCT_5036  62   
             FB_A5-Weekday-SDon-142000_B46_206 2015-01-06 MTA NYCT_4574  60   
             FB_A5-Weekday-SDon-142200_B41_77  2015-01-06 MTA NYCT_5127  35   
             FB_A5-Weekday-SDon-143200_B41_84  2015-01-06 MTA NYCT_4591   4   
                                                          MTA NYCT_4593  48   
             FB_A5-Weekday-SDon-143300_B41_83  2015-01-06 MTA NYCT_5052  58   
                                                          MTA NYCT_5127   1   
             FB_A5-Weekday-SDon-144400_B46_203 2015-01-06 MTA NYCT_5010  46   
             FB_A5-Weekday-SDon-144800_B46_204 2015-01-06 MTA NYCT_4962  31   
             FB_A5-Weekday-SDon-145600_B46_20

In [24]:
stop_distances = bustime.reset_index().drop_duplicates(['shape_id','next_stop_id','stop_dist_on_trip'])
stop_distances.shape_id = stop_distances.shape_id.str.replace('MTA_','')
stop_distances.next_stop_id = stop_distances.next_stop_id.str.replace('MTA_','')
stop_distances.drop(['route','trip_id','trip_date','vehicle_id','timestamp','lat','lon','dist_from_stop','presentable_dist','veh_dist_along_trip'],axis=1,inplace=True)

In [25]:
# verify that shape_id and next_stop_id is a valid primary key (that is, exactly one value for each index)
sum(stop_distances.duplicated(['shape_id','next_stop_id']))

0

In [26]:
stop_distances = stop_distances.sort(['shape_id','stop_dist_on_trip']).reset_index(drop=True)
stop_distances.head()

,shape_id,next_stop_id,stop_dist_on_trip
0,B410227,303270,334.47
1,B410227,303274,950.80
2,B410227,303276,1283.55
3,B410227,303277,1472.70
4,B410227,303222,2087.51


In [27]:
# mean distance between stops on trips
stop_distances.groupby('shape_id')['stop_dist_on_trip'].diff().mean()

375.39760814249365

## Identify interruptions as speed below threshold for some minimum duration

In [41]:
bustime['seg_speed'] = bg['veh_dist_along_trip'].diff()/(bg['timestamp'].diff()/ttools.datetime.timedelta(seconds=1))
bustime['seg_duration'] = bg['timestamp'].diff()/ttools.datetime.timedelta(seconds=1)

In [48]:
bustime.head(40)

lat  \
route        trip_id                         trip_date  vehicle_id                 
MTA NYCT_B41 FB_A5-Weekday-SDon-004500_B41_1 2015-01-06 MTA NYCT_4233  40.613280   
                                                        MTA NYCT_4233  40.613280   
                                                        MTA NYCT_4233  40.613280   
                                                        MTA NYCT_4233  40.610833   
                                                        MTA NYCT_4233  40.611432   
                                                        MTA NYCT_4233  40.612922   
                                                        MTA NYCT_4233  40.614282   
                                                        MTA NYCT_4233  40.617779   
                                                        MTA NYCT_4233  40.620874   
                                                        MTA NYCT_4233  40.622216   
                                                        MTA NYCT_4233  40.624716   
                                                        MTA NYCT_4233  40.627214   
                                                        MTA NYCT_4233  40.627758   
                                                        MTA NYCT_4233  40.630890   
                                                        MTA NYCT_4233  40.632639   
                                                        MTA NYCT_4233  40.632781   
                                                        MTA NYCT_4233  40.635526   
                                                        MTA NYCT_4233  40.637535   
                                                        MTA NYCT_4233  40.640351   
                                                        MTA NYCT_4233  40.642112   
                                                        MTA NYCT_4233  40.644067   
                                                        MTA NYCT_4233  40.644723   
                                                        MTA NYCT_4233  40.646539   
                                                        MTA NYCT_4233  40.650480   
                                                        MTA NYCT_4233  40.653385   
                                                        MTA NYCT_4233  40.655210   
                                                        MTA NYCT_4233  40.658258   
                                                        MTA NYCT_4233  40.659988   
                                                        MTA NYCT_4233  40.661865   
                                                        MTA NYCT_4233  40.662940   
                                                        MTA NYCT_4233  40.666451   
                                                        MTA NYCT_4233  40.669747   
                                                        MTA NYCT_4233  40.672709   
                                                        MTA NYCT_4233  40.673025   
                                                        MTA NYCT_4233  40.676708   
                                                        MTA NYCT_4233  40.679220   
                                                        MTA NYCT_4233  40.680226   
                                                        MTA NYCT_4233  40.683093   
                                                        MTA NYCT_4233  40.686209   
                                                        MTA NYCT_4233  40.688693   

                                                                             lon  \
route        trip_id                         trip_date  vehicle_id                 
MTA NYCT_B41 FB_A5-Weekday-SDon-004500_B41_1 2015-01-06 MTA NYCT_4233 -73.926553   
                                                        MTA NYCT_4233 -73.926553   
                                                        MTA NYCT_4233 -73.926553   
                                                        MTA NYCT_4233 -73.923459   
                                            

In [60]:
import numpy as np
bustime['interruption_index'] = np.nan
bustime.set_index('timestamp',append=True,inplace=True)
bustime.sort_index(inplace=True)

In [66]:
def index_interrupts(df):
    counter = 0
    interruption = False
    for index, row in df.iterrows():
        if row.seg_speed > 0.4:
            interruption = False
        else:
            if interruption is False:
                counter += 1
            else:
                pass
            interruption = True
            df.loc[index,'interruption_index'] = counter
    return df

In [67]:
bustime.groupby(level=(0,1,2,3)).apply(index_interrupts)

lat  \
route        trip_id                           trip_date  vehicle_id    timestamp                    
MTA NYCT_B41 FB_A5-Weekday-SDon-004500_B41_1   2015-01-06 MTA NYCT_4233 0 days 00:39:55  40.613280   
                                                                        0 days 00:40:58  40.613280   
                                                                        0 days 00:42:02  40.613280   
                                                                        0 days 00:45:41  40.610833   
                                                                        0 days 00:46:43  40.611432   
                                                                        0 days 00:47:45  40.612922   
                                                                        0 days 00:48:49  40.614282   
                                                                        0 days 00:49:52  40.617779   
                                                                        0 days 00:50:55  40.620874   
                                                                        0 days 00:51:57  40.622216   
                                                                        0 days 00:53:01  40.624716   
                                                                        0 days 00:54:04  40.627214   
                                                                        0 days 00:54:36  40.627758   
                                                                        0 days 00:55:39  40.630890   
                                                                        0 days 00:56:42  40.632639   
                                                                        0 days 00:57:45  40.632781   
                                                                        0 days 00:58:48  40.635526   
                                                                        0 days 00:59:52  40.637535   
                                                                        0 days 01:00:55  40.640351   
                                                                        0 days 01:01:58  40.642112   
                                                                        0 days 01:03:01  40.644067   
                                                                        0 days 01:03:33  40.644723   
                                                                        0 days 01:04:35  40.646539   
                                                                        0 days 01:05:39  40.650480   
                                                                        0 days 01:06:41  40.653385   
                                                                        0 days 01:07:46  40.655210   
                                                                        0 days 01:08:49  40.658258   
                                                                        0 days 01:09:52  40.659988   
                                                                        0 days 01:10:56  40.661865   
                                                                        0 days 01:11:58  40.662940   
...                                                                                            ...   
             FB_A5-Weekday-SDon-150500_B46_210 2015-01-06 MTA NYCT_5141 1 days 01:34:52  40.656658   
                                                                        1 days 01:35:55  40.655820   
                                                                        1 days 01:36:59  40.653756   
                                                                        1 days 01:37:30  40.653793   
                                                                        1 days 01:38:34  40.652392   
                                                                        1 days 01:39:37  40.651449   
                                                                        1 days 01:40:40  40.649297   
               